In [ ]:
import json
import os

import importlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import seaborn as sns
from sklearn import metrics
import torch

from models import avg_speed, schedule, ff, persistent, rnn, transformer, conv
from utils import data_utils, data_loader, model_utils, shape_utils

%matplotlib inline

# Get list of available train/test files for chosen run folder
run_folder_kcm = "../results/debug/"
run_folder_atb = "../results/debug/"
run_folder_tte = "../results/debug/"
kcm_data_folder = f"{run_folder_kcm}kcm/deeptte_formatted/"
atb_data_folder = f"{run_folder_atb}atb/deeptte_formatted/"
train_file_list_kcm = list(filter(lambda x: x[:5]=="train" and len(x)==6, os.listdir(kcm_data_folder)))
test_file_list_kcm = list(filter(lambda x: x[:4]=="test" and len(x)==5, os.listdir(kcm_data_folder)))
train_file_list_atb = list(filter(lambda x: x[:5]=="train" and len(x)==6, os.listdir(atb_data_folder)))
test_file_list_atb = list(filter(lambda x: x[:4]=="test" and len(x)==5, os.listdir(atb_data_folder)))

In [ ]:
# Select device to train on, and number workers if GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    NUM_WORKERS = 8
# elif torch.backends.mps.is_available():
#     device = torch.device("mps")
else:
    device = torch.device("cpu")
    NUM_WORKERS = 0
print(f"DEVICE: {device}")
print(f"WORKERS: {NUM_WORKERS}")

# Set hyperparameters
HIDDEN_SIZE = 32
BATCH_SIZE = 512
FOLD_MODEL = 0

# Define embedded variables for network models
embed_dict = {
    'timeID': {
        'vocab_size': 1440,
        'embed_dims': 24
    },
    'weekID': {
        'vocab_size': 7,
        'embed_dims': 4
    }
}

In [ ]:
# Load KCM Models

In [ ]:
# Declare baseline models
avg_model = data_utils.load_pkl(f"{run_folder_kcm}kcm/models/AVG_{FOLD_MODEL}.pkl")
sch_model = data_utils.load_pkl(f"{run_folder_kcm}kcm/models/SCH_{FOLD_MODEL}.pkl")
tim_model = data_utils.load_pkl(f"{run_folder_kcm}kcm/models/PER_TIM_{FOLD_MODEL}.pkl")

# Declare network models
ff_model = ff.FF(
    "FF",
    n_features=11,
    hidden_size=HIDDEN_SIZE,
    batch_size=BATCH_SIZE,
    embed_dict=embed_dict,
    device=device
).to(device)
ff_grid_model1 = ff.FF_GRID(
    "FF_GRID_IND",
    n_features=11,
    n_grid_features=8*3*3,
    hidden_size=HIDDEN_SIZE,
    grid_compression_size=8,
    batch_size=BATCH_SIZE,
    embed_dict=embed_dict,
    device=device
).to(device)
ff_grid_model2 = ff.FF_GRID_ATTN(
    "FF_GRID_ATTN",
    n_features=11,
    n_grid_features=8*3*3,
    n_channels=8,
    hidden_size=HIDDEN_SIZE,
    grid_compression_size=8,
    batch_size=BATCH_SIZE,
    embed_dict=embed_dict,
    device=device
).to(device)
ff_grid_model3 = ff.FF_GRID(
    "FF_NGRID_IND",
    n_features=11,
    n_grid_features=4*3*3*3,
    hidden_size=HIDDEN_SIZE,
    grid_compression_size=8,
    batch_size=BATCH_SIZE,
    embed_dict=embed_dict,
    device=device
).to(device)
gru_model = rnn.GRU(
    "GRU",
    n_features=8,
    hidden_size=HIDDEN_SIZE,
    batch_size=BATCH_SIZE,
    embed_dict=embed_dict,
    device=device
).to(device)
gru_grid_model1 = rnn.GRU_GRID(
    "GRU_GRID_IND",
    n_features=8,
    n_grid_features=8*3*3,
    hidden_size=HIDDEN_SIZE,
    grid_compression_size=8,
    batch_size=BATCH_SIZE,
    embed_dict=embed_dict,
    device=device
).to(device)
gru_grid_model2 = rnn.GRU_GRID_ATTN(
    "GRU_GRID_ATTN",
    n_features=8,
    n_grid_features=8*3*3,
    n_channels=8,
    hidden_size=HIDDEN_SIZE,
    grid_compression_size=8,
    batch_size=BATCH_SIZE,
    embed_dict=embed_dict,
    device=device
).to(device)
gru_grid_model3 = rnn.GRU_GRID(
    "GRU_NGRID_IND",
    n_features=8,
    n_grid_features=4*3*3*3,
    hidden_size=HIDDEN_SIZE,
    grid_compression_size=8,
    batch_size=BATCH_SIZE,
    embed_dict=embed_dict,
    device=device
).to(device)
trs_model = transformer.TRSF(
    "TRSF_ENC",
    n_features=8,
    hidden_size=HIDDEN_SIZE,
    batch_size=BATCH_SIZE,
    embed_dict=embed_dict,
    device=device
).to(device)

# Add all models to results list
base_model_list = []
base_model_list.append(avg_model)
base_model_list.append(sch_model)
base_model_list.append(tim_model)
nn_model_list = []
nn_model_list.append(ff_model)
nn_model_list.append(ff_grid_model1)
nn_model_list.append(ff_grid_model2)
nn_model_list.append(ff_grid_model3)
nn_model_list.append(gru_model)
nn_model_list.append(gru_grid_model1)
nn_model_list.append(gru_grid_model2)
nn_model_list.append(gru_grid_model3)
# nn_model_list.append(gru_mto_model)
# nn_model_list.append(conv1d_model)
nn_model_list.append(trs_model)
all_model_list = []
all_model_list.extend(base_model_list)
all_model_list.extend(nn_model_list)

# Load all model weights
for m in nn_model_list:
    m = m.load_state_dict(torch.load(f"{run_folder_kcm}kcm/models/{m.model_name}_{FOLD_MODEL}.pt"))

print(f"Model names: {[m.model_name for m in nn_model_list]}")
print(f"Model total parameters: {[sum(p.numel() for p in m.parameters()) for m in nn_model_list]}")

In [ ]:
print(f"Evaluating KCM on KCM")
model_fold_results = {}
for x in all_model_list:
    model_fold_results[x.model_name] = {"Labels":[], "Preds":[]}

for valid_file in test_file_list_kcm:
    # These are fold holdouts, separate validation files are used for generalization
    valid_data, grid, ngrid = data_utils.load_all_data(kcm_data_folder, valid_file)
    print(f"Successfully loaded {len(valid_data)} testing samples.")
    grid_content = grid.get_fill_content()
    print(f"VALIDATE FILE: {valid_file}")
    with open(f"{kcm_data_folder}train_config.json", "r") as f:
        config = json.load(f)

    # Construct dataloaders for all models
    dataloaders = []
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.basic_collate, NUM_WORKERS))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.basic_collate, NUM_WORKERS))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.sequential_collate, NUM_WORKERS))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.basic_collate, NUM_WORKERS))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.basic_grid_collate, NUM_WORKERS, grid=grid_content, buffer=1))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.basic_grid_collate, NUM_WORKERS, grid=grid_content, buffer=1))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.basic_grid_collate, NUM_WORKERS, grid=ngrid, is_ngrid=True, buffer=1))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.sequential_collate, NUM_WORKERS))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.sequential_grid_collate, NUM_WORKERS, grid=grid_content, buffer=1))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.sequential_grid_collate, NUM_WORKERS, grid=grid_content, buffer=1))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.sequential_grid_collate, NUM_WORKERS, grid=ngrid, is_ngrid=True, buffer=1))
    # dataloaders.append(data_loader.make_generic_dataloader(test_data, config, BATCH_SIZE, data_loader.sequential_mto_collate, NUM_WORKERS))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.transformer_collate, NUM_WORKERS))

    # Test all models
    for model, loader in zip(all_model_list, dataloaders):
        print(f"Evaluating: {model.model_name}")
        labels, preds = model.evaluate(loader, config)
        model_fold_results[model.model_name]["Labels"].extend(list(labels))
        model_fold_results[model.model_name]["Preds"].extend(list(preds))
kcm_kcm_model_fold_results = model_fold_results

print(f"Evaluating KCM on ATB")
model_fold_results = {}
for x in all_model_list:
    model_fold_results[x.model_name] = {"Labels":[], "Preds":[]}

for valid_file in test_file_list_atb:
    # These are fold holdouts, separate validation files are used for generalization
    valid_data, grid, ngrid = data_utils.load_all_data(atb_data_folder, valid_file)
    print(f"Successfully loaded {len(valid_data)} testing samples.")
    grid_content = grid.get_fill_content()
    print(f"VALIDATE FILE: {valid_file}")
    with open(f"{kcm_data_folder}train_config.json", "r") as f:
        config = json.load(f)

    # Construct dataloaders for all models
    dataloaders = []
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.basic_collate, NUM_WORKERS))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.basic_collate, NUM_WORKERS))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.sequential_collate, NUM_WORKERS))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.basic_collate, NUM_WORKERS))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.basic_grid_collate, NUM_WORKERS, grid=grid_content, buffer=1))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.basic_grid_collate, NUM_WORKERS, grid=grid_content, buffer=1))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.basic_grid_collate, NUM_WORKERS, grid=ngrid, is_ngrid=True, buffer=1))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.sequential_collate, NUM_WORKERS))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.sequential_grid_collate, NUM_WORKERS, grid=grid_content, buffer=1))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.sequential_grid_collate, NUM_WORKERS, grid=grid_content, buffer=1))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.sequential_grid_collate, NUM_WORKERS, grid=ngrid, is_ngrid=True, buffer=1))
    # dataloaders.append(data_loader.make_generic_dataloader(test_data, config, BATCH_SIZE, data_loader.sequential_mto_collate, NUM_WORKERS))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.transformer_collate, NUM_WORKERS))

    # Test all models
    for model, loader in zip(all_model_list, dataloaders):
        labels, preds = model.evaluate(loader, config)
        model_fold_results[model.model_name]["Labels"].extend(list(labels))
        model_fold_results[model.model_name]["Preds"].extend(list(preds))
kcm_atb_model_fold_results = model_fold_results


In [ ]:
# Load ATB Models

In [ ]:
# Declare baseline models
avg_model = data_utils.load_pkl(f"{run_folder_atb}atb/models/AVG_{FOLD_MODEL}.pkl")
sch_model = data_utils.load_pkl(f"{run_folder_atb}atb/models/SCH_{FOLD_MODEL}.pkl")
tim_model = data_utils.load_pkl(f"{run_folder_atb}atb/models/PER_TIM_{FOLD_MODEL}.pkl")

# Declare network models
ff_model = ff.FF(
    "FF",
    n_features=11,
    hidden_size=HIDDEN_SIZE,
    batch_size=BATCH_SIZE,
    embed_dict=embed_dict,
    device=device
).to(device)
ff_grid_model1 = ff.FF_GRID(
    "FF_GRID_IND",
    n_features=11,
    n_grid_features=8*3*3,
    hidden_size=HIDDEN_SIZE,
    grid_compression_size=8,
    batch_size=BATCH_SIZE,
    embed_dict=embed_dict,
    device=device
).to(device)
ff_grid_model2 = ff.FF_GRID_ATTN(
    "FF_GRID_ATTN",
    n_features=11,
    n_grid_features=8*3*3,
    n_channels=8,
    hidden_size=HIDDEN_SIZE,
    grid_compression_size=8,
    batch_size=BATCH_SIZE,
    embed_dict=embed_dict,
    device=device
).to(device)
ff_grid_model3 = ff.FF_GRID(
    "FF_NGRID_IND",
    n_features=11,
    n_grid_features=4*3*3*3,
    hidden_size=HIDDEN_SIZE,
    grid_compression_size=8,
    batch_size=BATCH_SIZE,
    embed_dict=embed_dict,
    device=device
).to(device)
gru_model = rnn.GRU(
    "GRU",
    n_features=8,
    hidden_size=HIDDEN_SIZE,
    batch_size=BATCH_SIZE,
    embed_dict=embed_dict,
    device=device
).to(device)
gru_grid_model1 = rnn.GRU_GRID(
    "GRU_GRID_IND",
    n_features=8,
    n_grid_features=8*3*3,
    hidden_size=HIDDEN_SIZE,
    grid_compression_size=8,
    batch_size=BATCH_SIZE,
    embed_dict=embed_dict,
    device=device
).to(device)
gru_grid_model2 = rnn.GRU_GRID_ATTN(
    "GRU_GRID_ATTN",
    n_features=8,
    n_grid_features=8*3*3,
    n_channels=8,
    hidden_size=HIDDEN_SIZE,
    grid_compression_size=8,
    batch_size=BATCH_SIZE,
    embed_dict=embed_dict,
    device=device
).to(device)
gru_grid_model3 = rnn.GRU_GRID(
    "GRU_NGRID_IND",
    n_features=8,
    n_grid_features=4*3*3*3,
    hidden_size=HIDDEN_SIZE,
    grid_compression_size=8,
    batch_size=BATCH_SIZE,
    embed_dict=embed_dict,
    device=device
).to(device)
trs_model = transformer.TRSF(
    "TRSF_ENC",
    n_features=8,
    hidden_size=HIDDEN_SIZE,
    batch_size=BATCH_SIZE,
    embed_dict=embed_dict,
    device=device
).to(device)

# Add all models to results list
base_model_list = []
base_model_list.append(avg_model)
base_model_list.append(sch_model)
base_model_list.append(tim_model)
nn_model_list = []
nn_model_list.append(ff_model)
nn_model_list.append(ff_grid_model1)
nn_model_list.append(ff_grid_model2)
nn_model_list.append(ff_grid_model3)
nn_model_list.append(gru_model)
nn_model_list.append(gru_grid_model1)
nn_model_list.append(gru_grid_model2)
nn_model_list.append(gru_grid_model3)
# nn_model_list.append(gru_mto_model)
# nn_model_list.append(conv1d_model)
nn_model_list.append(trs_model)
all_model_list = []
all_model_list.extend(base_model_list)
all_model_list.extend(nn_model_list)

# Load all model weights
for m in nn_model_list:
    m = m.load_state_dict(torch.load(f"{run_folder_atb}atb/models/{m.model_name}_{FOLD_MODEL}.pt"))

print(f"Model names: {[m.model_name for m in nn_model_list]}")
print(f"Model total parameters: {[sum(p.numel() for p in m.parameters()) for m in nn_model_list]}")

In [ ]:
print(f"Evaluating AtB on AtB")
model_fold_results = {}
for x in all_model_list:
    model_fold_results[x.model_name] = {"Labels":[], "Preds":[]}

for valid_file in test_file_list_atb:
    # These are fold holdouts, separate validation files are used for generalization
    valid_data, grid, ngrid = data_utils.load_all_data(atb_data_folder, valid_file)
    print(f"Successfully loaded {len(valid_data)} testing samples.")
    grid_content = grid.get_fill_content()
    print(f"VALIDATE FILE: {valid_file}")
    with open(f"{atb_data_folder}train_config.json", "r") as f:
        config = json.load(f)

    # Construct dataloaders for all models
    dataloaders = []
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.basic_collate, NUM_WORKERS))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.basic_collate, NUM_WORKERS))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.sequential_collate, NUM_WORKERS))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.basic_collate, NUM_WORKERS))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.basic_grid_collate, NUM_WORKERS, grid=grid_content, buffer=1))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.basic_grid_collate, NUM_WORKERS, grid=grid_content, buffer=1))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.basic_grid_collate, NUM_WORKERS, grid=ngrid, is_ngrid=True, buffer=1))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.sequential_collate, NUM_WORKERS))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.sequential_grid_collate, NUM_WORKERS, grid=grid_content, buffer=1))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.sequential_grid_collate, NUM_WORKERS, grid=grid_content, buffer=1))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.sequential_grid_collate, NUM_WORKERS, grid=ngrid, is_ngrid=True, buffer=1))
    # dataloaders.append(data_loader.make_generic_dataloader(test_data, config, BATCH_SIZE, data_loader.sequential_mto_collate, NUM_WORKERS))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.transformer_collate, NUM_WORKERS))

    # Test all models
    for model, loader in zip(all_model_list, dataloaders):
        print(f"Evaluating: {model.model_name}")
        labels, preds = model.evaluate(loader, config)
        model_fold_results[model.model_name]["Labels"].extend(list(labels))
        model_fold_results[model.model_name]["Preds"].extend(list(preds))
atb_atb_model_fold_results = model_fold_results

print(f"Evaluating AtB on KCM")
model_fold_results = {}
for x in all_model_list:
    model_fold_results[x.model_name] = {"Labels":[], "Preds":[]}

for valid_file in test_file_list_kcm:
    # These are fold holdouts, separate validation files are used for generalization
    valid_data, grid, ngrid = data_utils.load_all_data(kcm_data_folder, valid_file)
    print(f"Successfully loaded {len(valid_data)} testing samples.")
    grid_content = grid.get_fill_content()
    print(f"VALIDATE FILE: {valid_file}")
    with open(f"{atb_data_folder}train_config.json", "r") as f:
        config = json.load(f)

    # Construct dataloaders for all models
    dataloaders = []
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.basic_collate, NUM_WORKERS))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.basic_collate, NUM_WORKERS))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.sequential_collate, NUM_WORKERS))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.basic_collate, NUM_WORKERS))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.basic_grid_collate, NUM_WORKERS, grid=grid_content, buffer=1))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.basic_grid_collate, NUM_WORKERS, grid=grid_content, buffer=1))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.basic_grid_collate, NUM_WORKERS, grid=ngrid, is_ngrid=True, buffer=1))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.sequential_collate, NUM_WORKERS))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.sequential_grid_collate, NUM_WORKERS, grid=grid_content, buffer=1))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.sequential_grid_collate, NUM_WORKERS, grid=grid_content, buffer=1))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.sequential_grid_collate, NUM_WORKERS, grid=ngrid, is_ngrid=True, buffer=1))
    # dataloaders.append(data_loader.make_generic_dataloader(test_data, config, BATCH_SIZE, data_loader.sequential_mto_collate, NUM_WORKERS))
    dataloaders.append(data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.transformer_collate, NUM_WORKERS))

    # Test all models
    for model, loader in zip(all_model_list, dataloaders):
        labels, preds = model.evaluate(loader, config)
        model_fold_results[model.model_name]["Labels"].extend(list(labels))
        model_fold_results[model.model_name]["Preds"].extend(list(preds))
atb_kcm_model_fold_results = model_fold_results


In [ ]:
models = []
train_nets = []
test_nets = []
errors = []
results = kcm_kcm_model_fold_results
for model_name in results.keys():
    models.append(model_name)
    train_nets.append("KCM")
    test_nets.append("KCM")
    errors.append(np.round(np.sqrt(metrics.mean_squared_error(results[model_name]['Labels'], results[model_name]['Preds'])), 2))
results = kcm_atb_model_fold_results
for model_name in results.keys():
    models.append(model_name)
    train_nets.append("KCM")
    test_nets.append("ATB")
    errors.append(np.round(np.sqrt(metrics.mean_squared_error(results[model_name]['Labels'], results[model_name]['Preds'])), 2))
results = atb_atb_model_fold_results
for model_name in results.keys():
    models.append(model_name)
    train_nets.append("ATB")
    test_nets.append("ATB")
    errors.append(np.round(np.sqrt(metrics.mean_squared_error(results[model_name]['Labels'], results[model_name]['Preds'])), 2))
results = atb_kcm_model_fold_results
for model_name in results.keys():
    models.append(model_name)
    train_nets.append("ATB")
    test_nets.append("KCM")
    errors.append(np.round(np.sqrt(metrics.mean_squared_error(results[model_name]['Labels'], results[model_name]['Preds'])), 2))

metric_results = pd.DataFrame(columns=["Model","Train Network","Test Network","Metric"])
metric_results['Model'] = models
metric_results['Train Network'] = train_nets
metric_results['Test Network'] = test_nets
metric_results['Metric'] = errors
metric_results['Model-Train-Test'] = metric_results['Model']+"_"+metric_results['Train Network']+"_"+metric_results['Test Network']
metric_results = metric_results.sort_values('Model-Train-Test')
metric_results

In [ ]:
# Include DeepTTE Generalization Results
kcm_deeptte_gen_res = data_utils.extract_deeptte_results("KCM", run_folder_tte, "kcm/", generalization_flag=True)
atb_deeptte_gen_res = data_utils.extract_deeptte_results("ATB", run_folder_tte, "atb/", generalization_flag=True)
deeptte_gen_res = pd.concat([kcm_deeptte_gen_res, atb_deeptte_gen_res])
deeptte_gen_res["Model"] = "DEEPTTE"
deeptte_gen_res["Train Network"] = deeptte_gen_res["City"]
deeptte_gen_res["Test Network"] = deeptte_gen_res["Loss Set"]
deeptte_gen_res["Metric"] = deeptte_gen_res["RMSE"]
deeptte_gen_res = deeptte_gen_res[["Model","Train Network","Test Network","Metric"]]
deeptte_gen_res['Model-Train-Test'] = deeptte_gen_res['Model']+"_"+deeptte_gen_res['Train Network']+"_"+deeptte_gen_res['Test Network']
metric_results = pd.concat([metric_results, deeptte_gen_res])
metric_results = metric_results.sort_values('Model-Train-Test')
metric_results

In [ ]:
plot_data = metric_results[metric_results['Test Network']=="KCM"]
fig, axes = plt.subplots(1,1)
fig.set_figheight(6)
fig.set_figwidth(6)
sns.barplot(plot_data, x="Metric", y="Model-Train-Test", hue="Model", dodge=False)
axes.set_ylabel("Model")
axes.set_xlabel("RMSE")
fig.suptitle('Model Generalization Performance On KCM', fontsize=16)
fig.tight_layout()
plt.savefig("../plots/model_generalization.png", dpi=1800, bbox_inches='tight')

In [ ]:
plot_data = metric_results[metric_results['Test Network']=="ATB"]
fig, axes = plt.subplots(1,1)
fig.set_figheight(6)
fig.set_figwidth(6)
sns.barplot(plot_data, x="Metric", y="Model-Train-Test", hue="Model", dodge=False)
axes.set_ylabel("Model")
axes.set_xlabel("RMSE")
fig.suptitle('Model Generalization Performance On ATB', fontsize=16)
fig.tight_layout()
plt.savefig("../plots/model_generalization.png", dpi=1800, bbox_inches='tight')